In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torchvision
from torchvision import transforms
import cv2
import pickle

from I2L_140K import I2L_140K

In [2]:
train_set = I2L_140K("../data/dataset5")

train_load = DataLoader(dataset=train_set,
                        batch_size=64,
                        num_workers=4,
                        shuffle=True)

In [3]:
idx, (img, label) = next(enumerate(train_load))

print(img.shape)

torch.Size([64, 1, 128, 1088])
